# Collecte, Traitement, et Analyse de données de réseaux sociaux

Ouverture du fichier Json

In [ ]:
import json

with open('versailles_tweets_100.json') as json_data:
    data_dict = json.load(json_data)

print(data_dict)


Stocker l'information dans un DataFrame

In [108]:
import pandas as pd
df = pd.DataFrame(data_dict)
df

,_id,public_metrics,id,conversation_id,author_id,text,geo,lang,created_at,entities,context_annotations,attachments
0,1421616335700824064,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421616335700824064,1421616335700824064,1339914264522461187,Goumin des éléphants joueurs la même fatigue m...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T23:38:41.000Z,"{'hashtags': [{'start': 52, 'end': 63, 'tag': ...",NaN,NaN
1,1421599703116943360,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421599703116943360,1421599703116943360,1339914264522461187,@ericbailly24 @maxigr04del mes tontons vous a...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:32:35.000Z,"{'mentions': [{'start': 0, 'end': 13, 'usernam...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN
2,1421599163561742339,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1421599163561742339,1421599163561742339,1339914264522461187,Ah oui le sommeil là sera compliqué. #CIV est...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:30:27.000Z,"{'hashtags': [{'start': 37, 'end': 41, 'tag': ...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN
3,1421591889095057416,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421591889095057416,1421591889095057416,1339914264522461187,"31 juillet , journée internationale de la femm...",{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:01:32.000Z,"{'hashtags': [{'start': 60, 'end': 65, 'tag': ...",NaN,NaN
4,1421582795294617605,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421582795294617605,1421582795294617605,717025418,Le pedigree 😂😂😂😂 https://t.co/D3Rv7A2BOF,{'place_id': '00b8943291443c8c'},es,2021-07-31T21:25:24.000Z,"{'urls': [{'start': 17, 'end': 40, 'url': 'htt...",NaN,NaN
5,1421581383454052359,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",1421581383454052359,1421256177359466498,992904738516717570,@isabelle170516 @leonna_julie @Steiner2502 Vou...,{'place_id': '011a304d683c2461'},fr,2021-07-31T21:19:48.000Z,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,NaN
6,1421575939700445184,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421575939700445184,1421256177359466498,992904738516717570,@LynLyna12 @leonna_julie La grande muette cont...,{'place_id': '011a304d683c2461'},fr,2021-07-31T20:58:10.000Z,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",NaN,NaN
7,1421569996858269697,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421569996858269697,1421569996858269697,736523371,Under wsh 😳😳😳,{'place_id': '011a304d683c2461'},en,2021-07-31T20:34:33.000Z,NaN,NaN,NaN
8,1421563798503272448,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421563798503272448,1421563798503272448,1471684208,Les bains d'apollon #versailles #nocturne #app...,"{'place_id': '011a304d683c2461', 'coordinates'...",fr,2021-07-31T20:09:55.000Z,"{'urls': [{'start': 76, 'end': 99, 'url': 'htt...",NaN,NaN
9,1421562928176803848,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1421562928176803848,1421553997811421184,992904738516717570,@leonna_julie Le rdv aujourd'hui aura tenu se...,{'place_id': '011a304d683c2461'},fr,2021-07-31T20:06:28.000Z,"{'mentions': [{'start': 0, 'end': 13, 'usernam...",NaN,{'media_keys': ['3_1421562925999869960']}


Effacer tous les caractères spéciaux dans le texte

In [109]:
import re
df["texte_corrigé"] = df['text'].apply(lambda x: re.findall(r'\w*[a-zA-Z]+\w*', x))
df

,_id,public_metrics,id,conversation_id,author_id,text,geo,lang,created_at,entities,context_annotations,attachments,texte_corrigé
0,1421616335700824064,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421616335700824064,1421616335700824064,1339914264522461187,Goumin des éléphants joueurs la même fatigue m...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T23:38:41.000Z,"{'hashtags': [{'start': 52, 'end': 63, 'tag': ...",NaN,NaN,"[Goumin, des, éléphants, joueurs, la, même, fa..."
1,1421599703116943360,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421599703116943360,1421599703116943360,1339914264522461187,@ericbailly24 @maxigr04del mes tontons vous a...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:32:35.000Z,"{'mentions': [{'start': 0, 'end': 13, 'usernam...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN,"[ericbailly24, maxigr04del, mes, tontons, vous..."
2,1421599163561742339,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1421599163561742339,1421599163561742339,1339914264522461187,Ah oui le sommeil là sera compliqué. #CIV est...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:30:27.000Z,"{'hashtags': [{'start': 37, 'end': 41, 'tag': ...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN,"[Ah, oui, le, sommeil, là, sera, compliqué, CI..."
3,1421591889095057416,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421591889095057416,1421591889095057416,1339914264522461187,"31 juillet , journée internationale de la femm...",{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:01:32.000Z,"{'hashtags': [{'start': 60, 'end': 65, 'tag': ...",NaN,NaN,"[juillet, journée, internationale, de, la, fem..."
4,1421582795294617605,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421582795294617605,1421582795294617605,717025418,Le pedigree 😂😂😂😂 https://t.co/D3Rv7A2BOF,{'place_id': '00b8943291443c8c'},es,2021-07-31T21:25:24.000Z,"{'urls': [{'start': 17, 'end': 40, 'url': 'htt...",NaN,NaN,"[Le, pedigree, https, t, co, D3Rv7A2BOF]"
5,1421581383454052359,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",1421581383454052359,1421256177359466498,992904738516717570,@isabelle170516 @leonna_julie @Steiner2502 Vou...,{'place_id': '011a304d683c2461'},fr,2021-07-31T21:19:48.000Z,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,NaN,"[isabelle170516, leonna_julie, Steiner2502, Vo..."
6,1421575939700445184,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421575939700445184,1421256177359466498,992904738516717570,@LynLyna12 @leonna_julie La grande muette cont...,{'place_id': '011a304d683c2461'},fr,2021-07-31T20:58:10.000Z,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",NaN,NaN,"[LynLyna12, leonna_julie, La, grande, muette, ..."
7,1421569996858269697,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421569996858269697,1421569996858269697,736523371,Under wsh 😳😳😳,{'place_id': '011a304d683c2461'},en,2021-07-31T20:34:33.000Z,NaN,NaN,NaN,"[Under, wsh]"
8,1421563798503272448,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421563798503272448,1421563798503272448,1471684208,Les bains d'apollon #versailles #nocturne #app...,"{'place_id': '011a304d683c2461', 'coordinates'...",fr,2021-07-31T20:09:55.000Z,"{'urls': [{'start': 76, 'end': 99, 'url': 'htt...",NaN,NaN,"[Les, bains, d, apollon, versailles, nocturne,..."
9,1421562928176803848,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1421562928176803848,1421553997811421184,992904738516717570,@leonna_julie Le rdv aujourd'hui aura tenu se...,{'place_id': '011a304d683c2461'},fr,2021-07-31T20:06:28.000Z,"{'mentions': [{'start': 0, 'end': 13, 'usernam...",NaN,{'media_keys': ['3_1421562925999869960']},"[leonna_julie, Le, rdv, aujourd, hui, aura, te..."


Identification de l'auteur

In [ ]:
df['id']

Extraction de la liste de hashtags de la publication

In [112]:
df['hashtag'] = df['text'].apply(lambda x: re.findall(r'\B#\w*[a-zA-Z]+\w*', x))  #trouvé sur internet
df

,_id,public_metrics,id,conversation_id,author_id,text,geo,lang,created_at,entities,context_annotations,attachments,texte_corrigé,hashtag
0,1421616335700824064,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421616335700824064,1421616335700824064,1339914264522461187,Goumin des éléphants joueurs la même fatigue m...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T23:38:41.000Z,"{'hashtags': [{'start': 52, 'end': 63, 'tag': ...",NaN,NaN,"[Goumin, des, éléphants, joueurs, la, même, fa...",[#twitter225]
1,1421599703116943360,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421599703116943360,1421599703116943360,1339914264522461187,@ericbailly24 @maxigr04del mes tontons vous a...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:32:35.000Z,"{'mentions': [{'start': 0, 'end': 13, 'usernam...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN,"[ericbailly24, maxigr04del, mes, tontons, vous...","[#SupportriceMazo, #domie, #CIV]"
2,1421599163561742339,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1421599163561742339,1421599163561742339,1339914264522461187,Ah oui le sommeil là sera compliqué. #CIV est...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:30:27.000Z,"{'hashtags': [{'start': 37, 'end': 41, 'tag': ...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN,"[Ah, oui, le, sommeil, là, sera, compliqué, CI...",[#CIV]
3,1421591889095057416,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421591889095057416,1421591889095057416,1339914264522461187,"31 juillet , journée internationale de la femm...",{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:01:32.000Z,"{'hashtags': [{'start': 60, 'end': 65, 'tag': ...",NaN,NaN,"[juillet, journée, internationale, de, la, fem...",[#jifa]
4,1421582795294617605,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421582795294617605,1421582795294617605,717025418,Le pedigree 😂😂😂😂 https://t.co/D3Rv7A2BOF,{'place_id': '00b8943291443c8c'},es,2021-07-31T21:25:24.000Z,"{'urls': [{'start': 17, 'end': 40, 'url': 'htt...",NaN,NaN,"[Le, pedigree, https, t, co, D3Rv7A2BOF]",[]
5,1421581383454052359,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",1421581383454052359,1421256177359466498,992904738516717570,@isabelle170516 @leonna_julie @Steiner2502 Vou...,{'place_id': '011a304d683c2461'},fr,2021-07-31T21:19:48.000Z,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,NaN,"[isabelle170516, leonna_julie, Steiner2502, Vo...",[]
6,1421575939700445184,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421575939700445184,1421256177359466498,992904738516717570,@LynLyna12 @leonna_julie La grande muette cont...,{'place_id': '011a304d683c2461'},fr,2021-07-31T20:58:10.000Z,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",NaN,NaN,"[LynLyna12, leonna_julie, La, grande, muette, ...",[]
7,1421569996858269697,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421569996858269697,1421569996858269697,736523371,Under wsh 😳😳😳,{'place_id': '011a304d683c2461'},en,2021-07-31T20:34:33.000Z,NaN,NaN,NaN,"[Under, wsh]",[]
8,1421563798503272448,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421563798503272448,1421563798503272448,1471684208,Les bains d'apollon #versailles #nocturne #app...,"{'place_id': '011a304d683c2461', 'coordinates'...",fr,2021-07-31T20:09:55.000Z,"{'urls': [{'start': 76, 'end': 99, 'url': 'htt...",NaN,NaN,"[Les, bains, d, apollon, versailles, nocturne,...","[#versailles, #nocturne, #appollon]"
9,1421562928176803848,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1421562928176803848,1421553997811421184,992904738516717570,@leonna_julie Le rdv aujourd'hui aura tenu se...,{'place_id': '011a304d683c2461'},fr,2021-07-31T20:06:28.000Z,"{'mentions': [{'start': 0, 'end': 13, 'usernam...",NaN,{'media_keys': ['3_1421562925999869960']},"[leonna_julie, Le, rdv, aujourd, hui, aura, te...",[]


Extraction des mentions

In [113]:
df['mentions'] = df['text'].apply(lambda x: re.findall(r'\B@\w*[a-zA-Z]+\w*', x))  #trouvé sur internet
df

,_id,public_metrics,id,conversation_id,author_id,text,geo,lang,created_at,entities,context_annotations,attachments,texte_corrigé,hashtag,mentions
0,1421616335700824064,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421616335700824064,1421616335700824064,1339914264522461187,Goumin des éléphants joueurs la même fatigue m...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T23:38:41.000Z,"{'hashtags': [{'start': 52, 'end': 63, 'tag': ...",NaN,NaN,"[Goumin, des, éléphants, joueurs, la, même, fa...",[#twitter225],[]
1,1421599703116943360,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421599703116943360,1421599703116943360,1339914264522461187,@ericbailly24 @maxigr04del mes tontons vous a...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:32:35.000Z,"{'mentions': [{'start': 0, 'end': 13, 'usernam...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN,"[ericbailly24, maxigr04del, mes, tontons, vous...","[#SupportriceMazo, #domie, #CIV]","[@ericbailly24, @maxigr04del]"
2,1421599163561742339,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1421599163561742339,1421599163561742339,1339914264522461187,Ah oui le sommeil là sera compliqué. #CIV est...,{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:30:27.000Z,"{'hashtags': [{'start': 37, 'end': 41, 'tag': ...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN,"[Ah, oui, le, sommeil, là, sera, compliqué, CI...",[#CIV],[]
3,1421591889095057416,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421591889095057416,1421591889095057416,1339914264522461187,"31 juillet , journée internationale de la femm...",{'place_id': '00b8943291443c8c'},fr,2021-07-31T22:01:32.000Z,"{'hashtags': [{'start': 60, 'end': 65, 'tag': ...",NaN,NaN,"[juillet, journée, internationale, de, la, fem...",[#jifa],[]
4,1421582795294617605,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421582795294617605,1421582795294617605,717025418,Le pedigree 😂😂😂😂 https://t.co/D3Rv7A2BOF,{'place_id': '00b8943291443c8c'},es,2021-07-31T21:25:24.000Z,"{'urls': [{'start': 17, 'end': 40, 'url': 'htt...",NaN,NaN,"[Le, pedigree, https, t, co, D3Rv7A2BOF]",[],[]
5,1421581383454052359,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",1421581383454052359,1421256177359466498,992904738516717570,@isabelle170516 @leonna_julie @Steiner2502 Vou...,{'place_id': '011a304d683c2461'},fr,2021-07-31T21:19:48.000Z,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,NaN,"[isabelle170516, leonna_julie, Steiner2502, Vo...",[],"[@isabelle170516, @leonna_julie, @Steiner2502]"
6,1421575939700445184,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421575939700445184,1421256177359466498,992904738516717570,@LynLyna12 @leonna_julie La grande muette cont...,{'place_id': '011a304d683c2461'},fr,2021-07-31T20:58:10.000Z,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",NaN,NaN,"[LynLyna12, leonna_julie, La, grande, muette, ...",[],"[@LynLyna12, @leonna_julie]"
7,1421569996858269697,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421569996858269697,1421569996858269697,736523371,Under wsh 😳😳😳,{'place_id': '011a304d683c2461'},en,2021-07-31T20:34:33.000Z,NaN,NaN,NaN,"[Under, wsh]",[],[]
8,1421563798503272448,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421563798503272448,1421563798503272448,1471684208,Les bains d'apollon #versailles #nocturne #app...,"{'place_id': '011a304d683c2461', 'coordinates'...",fr,2021-07-31T20:09:55.000Z,"{'urls': [{'start': 76, 'end': 99, 'url': 'htt...",NaN,NaN,"[Les, bains, d, apollon, versailles, nocturne,...","[#versailles, #nocturne, #appollon]",[]
9,1421562928176803848,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1421562928176803848,1421553997811421184,992904738516717570,@leonna_julie Le rdv aujourd'hui aura tenu se...,{'place_id': '011a304d683c2461'},fr,2021-07-31T20:06:28.000Z,"{'mentions': [{'start': 0, 'end': 13, 'usernam...",NaN,{'media_keys': ['3_1421562925999869960']},"[leonna_julie, Le, rdv, aujourd, hui, aura, te...",[],[@leonna_julie]


Analyse de sentiments de la publication

In [115]:
from textblob import TextBlob
